In [19]:
import numpy as np
import time
from combat import find_balance, fight
from unit import make_unit, stack
import pandas as pd

In [4]:
t0 = time.time()
A = stack(make_unit(u'Upiorny smok'), 87760)
B = stack(make_unit(u'Archanioł'), 60907)

result = fight(A, B, 1000)
print result
time.time()-t0

{u'Upiorny smok': [0, 0], u'Archanio\u0142': [1000, 38270.959]}


0.28235793113708496

In [13]:
def calculate_scores():
    units = data.values[:-5]
    scores = np.ones((units.shape[0], units.shape[0]))

    t0 = time.time()
    i = 0
    for u_idx in xrange(units.shape[0]):
        u_name = units[u_idx][0]
        for v_idx in xrange(u_idx + 1, units.shape[0]):
            cnt_u, cnt_v = find_balance(u_name, units[v_idx][0], 500)
            scores[u_idx, v_idx] = cnt_u / float(cnt_v)
            scores[v_idx, u_idx] = cnt_v / float(cnt_u)
            i += 1
            if not i % 100:
                print 'Done {} pairs in {:.2f}s'.format(i, time.time() - t0)

    return scores

In [14]:
scores = calculate_scores()

Done 100 pairs in 176.65s
Done 200 pairs in 374.78s
Done 300 pairs in 564.65s
Done 400 pairs in 776.71s
Done 500 pairs in 1011.46s
Done 600 pairs in 1222.46s
Done 700 pairs in 1410.67s
Done 800 pairs in 1617.94s
Done 900 pairs in 1819.78s
Done 1000 pairs in 2023.00s
Done 1100 pairs in 2201.83s
Done 1200 pairs in 2385.48s
Done 1300 pairs in 2565.66s
Done 1400 pairs in 2748.50s
Done 1500 pairs in 2945.25s
Done 1600 pairs in 3130.23s
Done 1700 pairs in 3340.54s
Done 1800 pairs in 3590.49s
Done 1900 pairs in 3804.86s
Done 2000 pairs in 3953.76s
Done 2100 pairs in 4108.32s
Done 2200 pairs in 4341.79s
Done 2300 pairs in 4591.52s
Done 2400 pairs in 4805.11s
Done 2500 pairs in 5026.91s
Done 2600 pairs in 5244.86s
Done 2700 pairs in 5415.59s
Done 2800 pairs in 5603.84s
Done 2900 pairs in 5824.38s
Done 3000 pairs in 6078.25s
Done 3100 pairs in 6299.86s
Done 3200 pairs in 6495.13s
Done 3300 pairs in 6696.98s
Done 3400 pairs in 6906.17s
Done 3500 pairs in 7146.12s
Done 3600 pairs in 7330.95s
Done 

In [15]:
np.savez('misc/scores.npz', scores)

In [20]:
a = np.load('misc/scores.npz')['arr_0']
cs = pd.DataFrame.from_records(a, columns=data.Singular.values[:-5])
cs.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))
cs

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,...,Rdzawy smok,Czarodziej,Strzelec,Niziołek,Chłop,Knurzy jeździec,Mumia,Nomad,Rozbójnik,Troll
0,Pikinier,1.000000,1.186791,1.575000,2.618750,2.729393,2.826498,4.635417,6.288770,8.598750,...,118.268456,13.988439,7.116788,1.088750,0.180505,1.450000,2.700000,2.865449,1.330049,6.522293
1,Halabardnik,0.842609,1.000000,1.054393,2.238261,2.389381,2.473771,3.940870,5.480000,6.326087,...,98.888889,11.934783,5.572174,0.814783,0.149565,1.254325,2.325581,2.500000,1.146010,5.749579
2,Łucznik,0.634921,0.948413,1.000000,1.560317,2.994444,3.091787,3.824603,5.351587,4.844444,...,140.825786,6.460224,4.874603,0.588235,0.056033,1.388889,2.440476,3.200000,1.124603,6.572529
3,Kusznik,0.381862,0.446775,0.640895,1.000000,1.907609,2.700543,1.868963,2.663043,2.763043,...,140.675891,4.739522,3.675543,0.326087,0.031348,0.788043,1.286940,2.840761,0.564381,5.473009
4,Gryf,0.366382,0.418519,0.333952,0.524217,1.000000,1.052879,1.655556,2.240456,1.532479,...,43.346999,3.956125,1.707977,0.187876,0.067236,0.523932,0.980912,1.068946,0.461538,2.437515
5,Gryf królewski,0.353795,0.404241,0.323437,0.370296,0.949777,1.000000,1.595089,2.013616,1.464732,...,41.917222,3.646205,1.274232,0.183259,0.064955,0.497321,0.949330,0.996429,0.433036,2.167196
6,Zbrojny,0.215730,0.253751,0.261465,0.535056,0.604027,0.626924,1.000000,1.366517,1.634831,...,26.145401,2.871910,1.263499,0.202247,0.039326,0.317287,0.590293,0.629792,0.286807,1.405435
7,Krzyżowiec,0.159014,0.182482,0.186860,0.375510,0.446338,0.496619,0.731788,1.000000,0.846429,...,19.979592,2.276190,1.130612,0.112765,0.029252,0.237705,0.427148,0.494057,0.207596,1.154063
8,Mnich,0.116296,0.158076,0.206422,0.361920,0.652538,0.682719,0.611684,1.181435,1.000000,...,32.536928,1.570817,1.170000,0.128866,0.010835,0.293166,0.383959,0.701790,0.235643,1.431367
9,Kapłan,0.098644,0.115184,0.196400,0.270800,0.343478,0.488283,0.494554,0.635195,0.838667,...,22.421749,1.477592,1.084667,0.103867,0.010183,0.167205,0.308008,0.404407,0.128011,0.880860


In [21]:
cs.to_csv('scores/scores.csv', encoding='utf-8')

In [60]:
castle_scores_ai_value_based = np.zeros((14, 141))

x_id = 0
for x in castle_units:
    u_id = 0
    for u in all_units:
        x_aiv = data['AI Value'].values[x_id]
        u_aiv = data['AI Value'].values[u_id]
        castle_scores_ai_value_based[x_id, u_id] = x_aiv / float(u_aiv)
        u_id += 1
    x_id += 1

In [61]:
np.savez('misc/castle_scores_ai_value_based.npz', castle_scores_ai_value_based)

In [18]:
b = np.load('misc/castle_scores_ai_value_based.npz')['arr_0']
csaiv = pd.DataFrame.from_records(b.T, columns=data.Singular.values[:14])
csaiv.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))
csaiv

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,1.000000,1.437500,1.575000,2.300000,4.387500,5.600000,5.562500,7.350000,6.062500,9.375000,24.325000,26.250000,62.737500,109.700000
1,Halabardnik,0.695652,1.000000,1.095652,1.600000,3.052174,3.895652,3.869565,5.113043,4.217391,6.521739,16.921739,18.260870,43.643478,76.313043
2,Łucznik,0.634921,0.912698,1.000000,1.460317,2.785714,3.555556,3.531746,4.666667,3.849206,5.952381,15.444444,16.666667,39.833333,69.650794
3,Kusznik,0.434783,0.625000,0.684783,1.000000,1.907609,2.434783,2.418478,3.195652,2.635870,4.076087,10.576087,11.413043,27.277174,47.695652
4,Gryf,0.227920,0.327635,0.358974,0.524217,1.000000,1.276353,1.267806,1.675214,1.381766,2.136752,5.544160,5.982906,14.299145,25.002849
5,Gryf królewski,0.178571,0.256696,0.281250,0.410714,0.783482,1.000000,0.993304,1.312500,1.082589,1.674107,4.343750,4.687500,11.203125,19.589286
6,Zbrojny,0.179775,0.258427,0.283146,0.413483,0.788764,1.006742,1.000000,1.321348,1.089888,1.685393,4.373034,4.719101,11.278652,19.721348
7,Krzyżowiec,0.136054,0.195578,0.214286,0.312925,0.596939,0.761905,0.756803,1.000000,0.824830,1.275510,3.309524,3.571429,8.535714,14.925170
8,Mnich,0.164948,0.237113,0.259794,0.379381,0.723711,0.923711,0.917526,1.212371,1.000000,1.546392,4.012371,4.329897,10.348454,18.094845
9,Kapłan,0.106667,0.153333,0.168000,0.245333,0.468000,0.597333,0.593333,0.784000,0.646667,1.000000,2.594667,2.800000,6.692000,11.701333


In [65]:
csaiv.to_csv('castle_scores/castle_scores_ai_value_based.csv', encoding='utf-8')

In [19]:
comparison = cs.values[:,1:] / csaiv.values[:,1:]
comparison[np.abs(comparison-1) < .2] = 1.
comparison[comparison == 1.] = ''
comp_df = pd.DataFrame.from_records(comparison, columns=data.Singular.values[:14])
comp_df.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))

comp_df

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,,,,,0.620636,0.506329,,,1.41835,,0.606061,0.615858,0.623053,0.587803
1,Halabardnik,1.21125,,,1.40217,0.784447,0.635008,,,1.43732,1.3312,0.757077,0.770777,0.778605,0.710315
2,Łucznik,,,,,,,,,1.25918,,,,,
3,Kusznik,,0.715675,,,,,0.772785,,,,1.28438,1.28438,1.27812,1.62499
4,Gryf,1.61125,1.27739,,,,,1.30742,1.32483,,1.36253,,,,
5,Gryf królewski,1.9775,1.5713,,,1.21225,,1.60899,1.53741,1.35608,1.22493,,1.22338,1.26091,
6,Zbrojny,1.205,,,1.29402,0.764868,0.620901,,,1.37505,1.20013,0.73994,0.755774,0.768566,0.694987
7,Krzyżowiec,,,,,0.754814,0.650442,,,,1.23427,0.789262,,,0.728986
8,Mnich,0.705044,0.666667,0.794171,,,0.737418,0.714286,,,0.771065,,,,
9,Kapłan,,0.751654,,,0.733783,,,,1.29691,,,,,


In [20]:
comp_df.to_csv('castle_scores/castle_comparison.csv', encoding='utf-8')